# Nivel 1: Secciones conceptuales (Markdown)

## 4.1 – Resumen de métricas finales

### Evaluación Final del Modelo

Se compararon dos configuraciones del modelo de Regresión Logística:
- Umbral estándar (0.5)
- Umbral ajustado (0.3)

El ajuste de umbral permitió incrementar significativamente el recall de la clase minoritaria (incumplimiento),
a costa de un aumento en los falsos positivos.

## 4.2 - Discusión de trade-offs

El uso de un umbral reducido es adecuado en escenarios donde el costo de no detectar un incumplimiento
es mayor que el costo de generar una falsa alerta.

## 4.3 - Recomendación final

### Recomendación

El modelo de Regresión Logística es adecuado como línea base para la detección temprana de riesgo crediticio.
Se recomienda su uso con umbral ajustado en contextos preventivos,
y como apoyo a sistemas de decisión humana.

## 4.4 - Conclusión global del proyecto

El proyecto permitió desarrollar un pipeline completo de Machine Learning,
abarcando desde el análisis exploratorio de datos hasta la simulación de
despliegue del modelo.

La Regresión Logística demostró ser una línea base sólida para la detección
de riesgo crediticio, especialmente al priorizar el recall de la clase
minoritaria mediante el ajuste del umbral de decisión.

El enfoque aplicado permite apoyar la toma de decisiones en contextos reales,
destacando la importancia de balancear métricas según el costo del error.


# Nivel 2: Celdas técnicas (implementación)

## Celda 4.1 – Cargar artefactos necesarios

In [1]:
import pandas as pd
import numpy as np
import joblib

from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    roc_auc_score
)

**Celda 4.2 – Cargar datos de test**

In [2]:
X_test = pd.read_parquet("../artifacts/X_test.parquet")
y_test = pd.read_parquet("../artifacts/y_test.parquet")["TARGET"]

X_test.shape, y_test.shape

((61503, 210), (61503,))

**Celda 4.3 – Cargar modelo entrenado**

In [5]:
import joblib

model = joblib.load("../artifacts/log_reg_model.joblib")
threshold = joblib.load("../artifacts/decision_threshold.joblib")

**Celda 4.4 – Evaluación con umbral 0.5**

In [6]:
y_prob = model.predict_proba(X_test)[:, 1]
y_pred_05 = (y_prob >= 0.5).astype(int)

print("=== Umbral 0.5 ===")
print(classification_report(y_test, y_pred_05))
print("ROC-AUC:", roc_auc_score(y_test, y_prob))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_05))

=== Umbral 0.5 ===
              precision    recall  f1-score   support

           0       0.94      0.59      0.73     56538
           1       0.11      0.57      0.18      4965

    accuracy                           0.59     61503
   macro avg       0.53      0.58      0.46     61503
weighted avg       0.87      0.59      0.68     61503

ROC-AUC: 0.6137333651525161
Confusion Matrix:
 [[33598 22940]
 [ 2128  2837]]


**Comentario 4.4**

### Evaluación con umbral 0.5
Se evaluó el modelo con el umbral estándar (0.5).  
Dado el desbalance de clases, este umbral tiende a privilegiar el desempeño en la clase mayoritaria, lo que puede disminuir la capacidad de detectar correctamente incumplimientos (clase 1).

**Celda 4.5 – Evaluación con umbral ajustado (0.3)**

In [7]:
threshold = 0.30
y_pred_03 = (y_prob >= threshold).astype(int)

print(f"=== Umbral {threshold} ===")
print(classification_report(y_test, y_pred_03))
print("ROC-AUC:", roc_auc_score(y_test, y_prob))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_03))

=== Umbral 0.3 ===
              precision    recall  f1-score   support

           0       0.95      0.04      0.07     56538
           1       0.08      0.98      0.15      4965

    accuracy                           0.11     61503
   macro avg       0.52      0.51      0.11     61503
weighted avg       0.88      0.11      0.08     61503

ROC-AUC: 0.6137333651525161
Confusion Matrix:
 [[ 2111 54427]
 [  101  4864]]


**Comentario 4.5**

### Evaluación con umbral ajustado (0.3)
Al reducir el umbral, el modelo aumenta la detección de la clase minoritaria (incumplimiento), mejorando el **recall** de la clase 1.  
Sin embargo, esto incrementa los falsos positivos, generando más alertas para clientes que finalmente no incumplen.

**Celda 4.6 – Comparación final de evaluación**

## Comparación final y recomendación

- **Umbral 0.5:** mejor desempeño general en la clase mayoritaria, pero menor capacidad de detectar incumplimientos.
- **Umbral 0.3:** mejor detección de incumplimientos (mayor recall clase 1), a costa de más falsos positivos.

**Recomendación:**  
El umbral ajustado es adecuado en contextos donde el costo de no detectar un incumplimiento es mayor que el costo de una falsa alerta.